## Para ejecutar el código
Es necesario generar un token de HuggingFace y guardarlo como secreto llamado `HF_TOKEN`

https://huggingface.co/docs/hub/security-tokens

A continuación instrucciones desde https://colab.research.google.com/github/google-health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb#scrollTo=qRFQnPL2a9Dj

### Authenticate with Hugging Face

Generate a Hugging Face `read` access token by going to [settings](https://huggingface.co/settings/tokens).

If you are using Google Colab, add your access token to the Colab Secrets manager to securely store it. If not, proceed to run the cell below to authenticate with Hugging Face.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

## Obtener acceso al modelo

Posteriormente ir a https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3/tree/main habiendo iniciado sesión con la misma cuenta que se usó para generar el token, y aceptar los términos y condiciones.


Listo! Eso es todo lo necesario para correr el código.

## Obtener acceso al modelo

Posteriormente ir a https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3/tree/main habiendo iniciado sesión con la misma cuenta que se usó para generar el token, y aceptar los términos y condiciones.


Listo! Eso es todo lo necesario para correr el código.

## Disclaimer:

El código se corrió usando una GPU L4 y esta optimizado para tal, se recomienda utilizar el mismo entorno para su ejecución

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True,max_split_size_mb:128

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True,max_split_size_mb:128


In [ ]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

DATASET_NAME = "dserranog/fewshot-narrative-examples"

dataset = load_dataset(DATASET_NAME)
dataset

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

epic_narrative_examples.json: 0.00B [00:00, ?B/s]

noir_narrative_examples.json: 0.00B [00:00, ?B/s]

sci_fi_narrative_examples.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/45 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['original', 'rewritten', 'style', 'source'],
        num_rows: 45
    })
})

In [ ]:
import pandas as pd

for split in dataset:
    print(f"\n📚 {split.upper()}")
    df = pd.DataFrame(dataset[split])
    display(df.sample(3))


📚 TRAIN


,original,rewritten,style,source
1,There were ravens flying around the old tower.,"Ravens circled the ruined tower, bearing secre...",epic,Game of Thrones
34,The house was quiet except for the computer.,The house was silent except for the constant h...,sci-fi,Black Mirror
4,Fire and ice met in battle.,"When fire and ice collide, only the ancient go...",epic,Game of Thrones


In [ ]:
from google.colab import userdata
from huggingface_hub import login
TOKEN_NAME = "HF_TOKEN"
hf_token = userdata.get(TOKEN_NAME)
if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in Colab Secrets.")

Successfully logged in to Hugging Face!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch
import gc

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def get_lora_model(base_model=False):
    torch.cuda.empty_cache()
    gc.collect()

    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",          # shards across GPU/CPU
        torch_dtype=torch.float16,  # internal compute in fp16
        low_cpu_mem_usage=True,
    )
    model.config.pad_token_id = tokenizer.pad_token_id # updating model config
    if base_model:
        return model
    # r: rank dimension for LoRA update matrices (smaller = more compression)
    rank_dimension = 8
    # lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
    lora_alpha = 16
    # lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
    lora_dropout = 0.05

    # LoRa config
    lora_config = LoraConfig(
        r=rank_dimension, # Rank dimension - typically between 4-32
        lora_alpha=lora_alpha, # LoRA scaling factor - typically 2x rank
        target_modules=["q_proj", "v_proj"],  # Which modules to apply LoRA to
        lora_dropout=lora_dropout, # Dropout probability for LoRA layers
        bias="none", # Bias type for LoRA. the corresponding biases will be updated during training.
        task_type="CAUSAL_LM", # Task type for model architecture
    )
    return get_peft_model(model, lora_config)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset(DATASET_NAME, split="train")
noir_dataset = dataset.filter(lambda example: example["style"] == "noir")
noir_dataset.to_pandas().head()

Filter:   0%|          | 0/45 [00:00<?, ? examples/s]

,original,rewritten,style,source
0,The city was very hot in the afternoon.,"It was a dry afternoon, and the sun lashed the...",noir,Chinatown
1,I could tell people here weren't honest.,I could smell the lies in the air before I eve...,noir,Chinatown
2,She looked at me with deep and troubled eyes.,"Her eyes were a stormy sea, hiding shipwrecks ...",noir,Chinatown
3,People here didn't trust each other.,"In this town, trust was the most dangerous gam...",noir,Chinatown
4,I smoked and watched the smoke rise.,I lit a cigarette and watched the smoke curl u...,noir,Chinatown


In [ ]:
from transformers import DataCollatorForLanguageModeling

noir_dataset = dataset.filter(lambda example: example["style"] == "noir")

# 1. Build prompt & response from your curated fields
def format_prompt(example):
    original = example["original"]
    rewritten = example["rewritten"]
    # insert the style dynamically
    example["prompt"]   = f"Rewrite the following text in {example['style']} style: '{original}'"
    example["response"] = rewritten
    return example

# apply to only the noir subset
noir_dataset = noir_dataset.map(
    format_prompt,
    remove_columns=["original", "rewritten", "style", "source"]
)

# 2. Prepare the collator (causal LM, so mlm=False)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,  # Optional: pad to multiple of 8 for tensor cores
)

# 3. Tokenize prompt + response into input_ids / attention_mask

def tokenize_function_with_eos(example):
    # Construct the full text with proper Mistral format
    full_text = f"<s>[INST]{example['prompt'].strip()}[/INST]{example['response'].strip()}</s>"

    # Tokenize the full text
    tokenized_full = tokenizer(
        full_text,
        truncation=True,
        padding=False,  # Let the data collator handle padding
        max_length=512,
        return_tensors=None  # Return lists, not tensors
    )

    # Create labels (copy of input_ids)
    tokenized_full["labels"] = tokenized_full["input_ids"].copy()

    # Find where the response starts by tokenizing just the prompt part
    prompt_part = f"<s>[INST]{example['prompt'].strip()}[/INST]"
    tokenized_prompt = tokenizer(
        prompt_part,
        truncation=True,
        max_length=512,
        return_tensors=None
    )
    prompt_length = len(tokenized_prompt["input_ids"])

    # Mask the prompt tokens in labels (we only want to learn from the response)
    tokenized_full["labels"][:prompt_length] = [-100] * prompt_length

    return tokenized_full

tokenized_noir_dataset = noir_dataset.map(
    tokenize_function_with_eos,
    remove_columns=["prompt", "response"]
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

lora_model_noir_eos = get_lora_model()
lora_model_noir_eos.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="/content/lora_noir_outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)
trainer = Trainer(
    model=lora_model_noir_eos,
    train_dataset=tokenized_noir_dataset,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/tmp/ipython-input-9-569990450.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470


Step,Training Loss
10,5.027000


TrainOutput(global_step=12, training_loss=4.923726161321004, metrics={'train_runtime': 29.9067, 'train_samples_per_second': 1.505, 'train_steps_per_second': 0.401, 'total_flos': 88139574411264.0, 'train_loss': 4.923726161321004, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

generator_noir = pipeline(
    "text-generation",
    model=lora_model_noir_eos,
    tokenizer=tokenizer,
)

# Format the prompt correctly with [INST] tokens
prompt = "[INST]Rewrite the following text in noir style (keep it concise): 'The phone rang at midnight.'[/INST]"

print("=== Generation after 3 epochs ===\n")
for i in range(4):
    output = generator_noir(
        prompt,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.55,
        top_p=0.78,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    generated_text = output[0]["generated_text"]

    # Extract just the response part (after [/INST])
    response = generated_text.split("[/INST]")[-1].strip()
    print(f"📝 Example {i+1}:\n{response}\n{'-'*60}\n")

Device set to use cuda:0


=== Generation after 3 epochs ===

📝 Example 1:
The midnight hour struck, and the phone began its mournful wail. A sound that could pierce the heart of even the hardest soul. I reached for it, my hand trembling slightly, knowing that the call could bring either good or bad news.
------------------------------------------------------------

📝 Example 2:
The midnight hour struck, and the phone began to wail like a mournful siren. A chilling echo in the silent, inky blackness of the room. The call was never a good sign.
------------------------------------------------------------

📝 Example 3:
The midnight hour struck, and the phone began its mournful wail. A sound that could only mean trouble in a city where the moon cast long, dark shadows.
------------------------------------------------------------

📝 Example 4:
The midnight hour struck, and the telephone began its mournful wail, echoing through the empty room. A chilling prelude to the secrets that would soon be unveiled.
-----------